In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial import distance
import numpy as np
import keras
import tensorflow as tf
import os
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from skimage.feature import hog

In [ ]:
BASE_DIR_PATH = '...'

In [ ]:
def create_img_df(df, img_scale):
    pil_imgs = []
    preprocessed_imgs = []
    flattened_imgs = []
    for i in range(len(df)):
        img_file = df['img_file'][i]
        img = image.load_img(img_file.strip(), target_size=(img_scale, img_scale), interpolation='lanczos')
        img_array = image.img_to_array(img)
        x = np.expand_dims(img_array, axis=0)
        x = preprocess_input(x)
        preprocessed_imgs.append(x)
        flat_img = x.flatten()
        flattened_imgs.append(flat_img)
        pil_imgs.append(img)
    pil_imgs = np.array(pil_imgs)
    preprocessed_imgs = np.array(preprocessed_imgs)
    flattened_imgs = np.array(flattened_imgs)
    return (pil_imgs, preprocessed_imgs, flattened_imgs)

In [ ]:
columns_names = ['img', 'label']

img_32_array = np.load(BASE_DIR_PATH + '', allow_pickle=True)
img_32_df = pd.DataFrame(img_32_array)

In [ ]:
# create HOG Dataframes
hog_img_32_array = []
img_scale = 32

for i in range(len(img_32_df)):
    img, label = img_32_df['img'][i], img_32_df['label'][i]
    hog_fd, hog_img = hog(
        image=np.array(preprocess_input).reshape(img_scale, img_scale,3),
        orientations=9,
        pixels_per_cell=(8,8),
        cells_per_block=(2,2),
        visualize=True,
        channel_axis=2
    )
    hog_img_32_array.append(img, hog_fd, hog_img, label)

hog_img_32_array = np.array(hog_img_32_array)

In [ ]:
hog_column_names = ['img', 'hog_fd', 'hog_img', 'label']

hog_img_32_df = pd.DataFrame(hog_img_32_array, columns=hog_column_names)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def get_model_scores(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    total_samples_per_class = cm.sum(axis=1)
    accuracies_per_class = cm.diagonal() / total_samples_per_class

    class_0_accuracy = accuracies_per_class[0]
    class_1_accuracy = accuracies_per_class[1]

    metrics = (acc, prec, recall, f1, roc, class_0_accuracy, class_1_accuracy)
    return metrics

In [ ]:
from sklearn.model_selection import StratifiedKFold

def k_fold_scores(model, df_img, df_hog_img, model_name=None, n_splits=10, seed=12172023) -> dict:

    random.seed(seed)

    rand_list = random.sample(range(0, len(df_img)), len(df_img))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    k_fold_results = {}

    # Vectorized images
    randomized_imgs = []

    # shuffle records according to rand_list
    for i in range(len(df_img)):
        ind = rand_list[i]
        flat_img, outcome = df_img['flat_img_array'][ind], df_img['outcome'][ind]
        randomized_imgs.append(np.array([flat_img, outcome]))

    randomized_imgs = np.array(randomized_imgs)

    X, y = randomized_imgs[:, 0], randomized_imgs[:, 1]

    wide_X = np.zeros((len(X), X[0].shape[0]))
    for indx in range(len(X)):
        for feature in range(X[0].shape[0]):
            wide_X[indx][feature] = X[indx][feature]
    
    y = y.astype(int)

    # Vectorized Stratified K-Fold
    acc_scores = []
    prec_scores = []
    recall_scores = []
    f1_scores = []
    roc_scores = []
    class_0_acc_scores = []
    class_1_acc_scores = []

    for train_indx, test_indx in skf.split(wide_X, y):
        X_train, X_test = wide_X[train_indx], wide_X[test_indx]
        y_train, y_test = y[train_indx], y[test_indx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc, prec, recall, f1, roc, class_0_accuracy, class_1_accuracy = get_model_scores(y_test, y_pred)
        acc_scores.append(acc); prec_scores.append(prec)
        recall_scores.append(recall); f1_scores.append(f1)
        roc_scores.append(roc); class_0_acc_scores.append(class_0_accuracy)
        class_1_acc_scores.append(class_1_accuracy)

    avg_acc = np.mean(acc_scores)
    avg_prec_scores = np.mean(prec_scores)
    avg_recall_scores = np.mean(recall_scores)
    avg_f1_scores = np.mean(f1_scores)
    avg_roc_scores = np.mean(roc_scores)
    avg_class_0_acc_scores = np.mean(class_0_acc_scores)
    avg_class_1_acc_scores = np.mean(class_1_acc_scores)

    if model_name is not None:
        print(f'(Vectorized) {model_name} Model Stratified {n_splits}-Fold Results:')
    print(f'{avg_acc}')
    print(f'{avg_prec_scores=}')
    print(f'{avg_recall_scores=}')
    print(f'{avg_f1_scores=}')
    print(f'{avg_roc_scores=}')
    print(f'{avg_class_0_acc_scores=}')
    print(f'{avg_class_1_acc_scores=}')

    k_fold_results['flat_img_results'] = (avg_acc, avg_prec_scores, avg_recall_scores, avg_f1_scores, avg_roc_scores, avg_class_0_acc_scores, avg_class_1_acc_scores)

    # HOG
    hog_records = []

    for i in range(len(df_hog_img)):
        ind = rand_list[i]
        hog_fd, outcome = df_hog_img['hog_fd'][ind], df_hog_img['outcome'][ind]
        hog_records.append(np.array([hog_fd, outcome]))

    hog_records = np.array(hog_records)

    hog_X, hog_y = hog_records[:, 0], hog_records[:, 1]

    hog_wide_X = np.zeros((len(hog_X), hog_X[0].shape))
    for indx in range(len(hog_X)):
        for feature in range(hog_X[0].shape[0]):
            hog_wide_X[indx][feature] = hog_X[indx][feature]

    # HOG Stratified K-Fold
    hog_acc_scores = []
    hog_prec_scores = []
    hog_recall_scores = []
    hog_f1_scores = []
    hog_roc_scores = []
    hog_class_0_acc_scores = []
    hog_class_1_acc_scores = []

    for train_indx, test_indx in skf.split(hog_wide_X, hog_y):
        X_train, X_test = hog_wide_X[train_indx], hog_wide_X[test_indx]
        y_train, y_test = hog_y[train_indx], hog_y[test_indx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc, prec, recall, f1, roc, class_0_accuracy, class_1_accuracy = get_model_scores(y_test, y_pred)
        hog_acc_scores.append(acc); hog_prec_scores.append(prec)
        hog_recall_scores.append(recall); hog_f1_scores.append(f1)
        hog_roc_scores.append(roc); hog_class_0_acc_scores.append(class_0_accuracy)
        hog_class_1_acc_scores.append(class_1_accuracy)
    
    hog_avg_acc = np.mean(hog_acc_scores)
    hog_avg_prec_scores = np.mean(hog_prec_scores)
    hog_avg_recall_scores = np.mean(hog_recall_scores)
    hog_avg_f1_scores = np.mean(hog_f1_scores)
    hog_avg_roc_scores = np.mean(hog_roc_scores)
    hog_avg_class_0_acc_scores = np.mean(hog_class_0_acc_scores)
    hog_avg_class_1_acc_scores = np.mean(hog_class_1_acc_scores)

    if model_name is not None:
        print(f'(HOG) {model_name} Model Stratified {n_splits}-Fold Results:')
    print(f'{hog_avg_acc}')
    print(f'{hog_avg_prec_scores=}')
    print(f'{hog_avg_recall_scores=}')
    print(f'{hog_avg_f1_scores=}')
    print(f'{hog_avg_roc_scores=}')
    print(f'{hog_avg_class_0_acc_scores=}')
    print(f'{hog_avg_class_1_acc_scores=}')

    k_fold_results['hog_results'] = (hog_avg_acc, hog_avg_prec_scores, hog_avg_recall_scores, hog_avg_f1_scores, hog_avg_roc_scores, hog_avg_class_0_acc_scores, hog_avg_class_1_acc_scores)

    return k_fold_results


In [ ]:
from datetime import datetime
def today_date():
    return f'{datetime.today().month}.{datetime.today().day}.{datetime.today().year}'

In [ ]:
def class_0_accuracy(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    total_samples_per_class = cm.sum(axis=1)
    accuracies_per_class = cm.diagonal() / total_samples_per_class
    class_0_accuracy = accuracies_per_class[0]
    return class_0_accuracy

def class_1_accuracy(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    total_samples_per_class = cm.sum(axis=1)
    accuracies_per_class = cm.diagonal() / total_samples_per_class
    class_1_accuracy = accuracies_per_class[1]
    return class_1_accuracy

In [ ]:
# Pipeline (HOG + SVM hyperparameter tuning)

import logging
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

class HOGTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=9, pixels_per_cell=(8,8),
                 cells_per_block=(2,2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        hog_X = []
        img_scale = X[0].shape[1]

        for x in X:
            hog_fd, _ = hog(
                image=np.array(x).reshape((img_scale, img_scale, 3)),
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block,
                visualize=True,
                channel_axis=2
            )
            hog_X.append(hog_fd)
        
        hog_X = np.array(hog_X)

        hog_wide_X = np.zeros((len(hog_X), hog_X[0].shape[0]))
        for indx in range(len(hog_X)):
            for feature in range(hog_X[0].shape):
                hog_wide_X[indx][feature] = hog_X[indx][feature]
        
        return hog_wide_X

In [ ]:
def hog_svm_hyperparameter_tester(img_df, param_grid, experiment_name, n_splits=3, seed=12172023):
    
    # preprocessed_img = PIL image
    X, y = np.array(img_df['preprocessed_img'][:]), np.array(img_df['outcome'][:]).astype(int)

    log_file = './experiment_logs/{}_{}'.format(today_date(), experiment_name)

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    file_handler = logging.FileHandler(log_file, mode='w')
    file_handler.setLevel(logging.INFO)

    formatter = logging.Formatter('%(asctime)s %(message)s')
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)

    random.seed(seed)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    scorers = {
        'roc_auc': make_scorer(roc_auc_score),
        'class_0_accuracy': make_scorer(class_0_accuracy),
        'class_1_accuracy': make_scorer(class_1_accuracy),
        'precision': make_scorer(precision_score),
        'accuracy': make_scorer(accuracy_score),
        'recall': make_scorer(recall_score),
        'f1': make_scorer(f1_score)
    }

    # pipeline
    pipeline = Pipeline([
        ('HOG', HOGTransformer()),
        ('SVM', SVC())
    ])

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=scorers,
        refit='roc_auc',
        cv=skf,
        verbose=2
    )

    grid_search.fit(X, y)

    cv_results = grid_search.cv_results_
    for i in range(len(cv_results['params'])):
        print('Parameters: {}'.format(cv_results['params'][i]))
        logging.info('Parameters: {}'.format(cv_results['params'][i]))
        for scorer in scorers:
            print('{}: {}'.format(scorer, cv_results[f'mean_test_{scorer}'][i]))
            logging.info('{}: {}'.format(scorer, cv_results[f'mean_test_{scorer}'][i]))
        print('\n'); logging.info('\n')
    
    print('Best parameters found: {}'.format(grid_search.best_params_))
    print('Best cross-validated ROC AUC Score: {}'.format(grid_search.best_score_))
    logging.info('Best parameters found: {}'.format(grid_search.best_params_))
    logging.info('Best cross-validated ROC AUC Score: {}'.format(grid_search.best_score_))

    try:
        file_handler.close()
    except Exception as e:
        print('Error shutting down the logging.\nError msg: {}'.format(e))
        pass

    return grid_search